In [2]:
import h5py
import pandas as pd
import numpy as np

In [3]:
id_tab = pd.read_csv("/tf/notebooks/schnemau/xAI_stroke_3d/data/10Fold_ids_V0.csv", sep=",")

In [28]:
dat_norm = pd.read_csv("/tf/notebooks/hezo/stroke_perfusion/data/baseline_data_zurich_prepared.csv", sep = ",")
dat_norm = dat_norm[dat_norm["p_id"].isin(id_tab["p_id"].values)]
dat_norm = dat_norm.reset_index(drop=True)

In [30]:
dat_og = pd.read_csv("/tf/notebooks/schnemau/xAI_stroke_3d/data/baseline_data_zurich_prepared0.csv", sep=";", decimal = ",")
dat_og = dat_og[dat_og["p_id"].isin(id_tab["p_id"].values)]
dat_og = dat_og.reset_index(drop=True)

In [31]:
print(dat_norm.shape)
print(dat_og.shape)

(407, 16)
(407, 18)


## Heatmap dependence plot

##### input
- specify patient and covariate
- specify heatmap type

#### outputs
- set of heatmaps
- set of assembled, data-frames with predictions

# Data generation

In [107]:
X = np.load("/tf/notebooks/schnemau/xAI_stroke_3d/data/prepocessed_dicom_3d.npy")
(X_train, X_valid, X_test),(X_tab_train, X_tab_valid, X_tab_test), (y_train, y_valid, y_test) = rdat.split_data_tabular(id_tab, X, which_split)

In [109]:
X_tab_test[0]

array([-0.09263432, -1.29584762, -0.16904841,  1.88154169,  2.37904752,
       -0.26359616, -0.11820485,  0.71268841,  2.22596351,  0.80633793,
        1.79405659, -0.46097582, -0.47349193])

In [130]:
dat_norm["age"].min()


-3.22279770649683

In [132]:
selected_covariate = "age"
dat_norm = pd.read_csv("/tf/notebooks/hezo/stroke_perfusion/data/baseline_data_zurich_prepared.csv", sep = ",")
values = np.linspace(dat_norm[selected_covariate].min(), dat_norm[selected_covariate].max(), num=40)





array([-3.22279771, -3.09684252, -2.97088734, -2.84493215, -2.71897697,
       -2.59302178, -2.4670666 , -2.34111141, -2.21515623, -2.08920105,
       -1.96324586, -1.83729068, -1.71133549, -1.58538031, -1.45942512,
       -1.33346994, -1.20751475, -1.08155957, -0.95560438, -0.8296492 ,
       -0.70369402, -0.57773883, -0.45178365, -0.32582846, -0.19987328,
       -0.07391809,  0.05203709,  0.17799228,  0.30394746,  0.42990265,
        0.55585783,  0.68181301,  0.8077682 ,  0.93372338,  1.05967857,
        1.18563375,  1.31158894,  1.43754412,  1.56349931,  1.68945449])

In [ ]:
# Assuming X_tab_test is a numpy array
covariate_index = 0  # Replace with the actual index of the covariate/column you want to analyze

# Generate PDP for the specified covariate
values = np.linspace(X_tab_test[:, covariate_index].min(), X_tab_test[:, covariate_index].max(), num=40)
partial_dependency_values = []

for value in values:
    modified_data = X_tab_test.copy()
    modified_data[:, covariate_index] = value
    modified_input = (X_test, modified_data)  # Keep 3D images constant, modify only covariate
    modified_input = tuple(tf.convert_to_tensor(data) for data in modified_input)
    
    # Predict with the modified input
    prediction = predict_ontram(model_3d, data=[modified_input])['pdf'][:, 1]
    partial_dependency_values.append(prediction.mean())  # Assuming you want to visualize average predictions

# Plot the Partial Dependency Plot
plt.plot(values, partial_dependency_values, marker='o')
plt.title(f'Partial Dependency Plot for Variable {covariate_index}')
plt.xlabel(f'Variable {covariate_index}')
plt.ylabel('Average Class 1 Probability')
plt.show()

# Prediction framework

In [65]:
%matplotlib inline

import os
import h5py
import pandas as pd
import numpy as np
import pickle as pkl
import time

import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import to_categorical
from statsmodels.stats.proportion import proportion_confint
from sklearn.metrics import confusion_matrix
from sklearn import metrics

import tensorflow as tf
from tensorflow import keras

print("TF  Version",tf.__version__)

TF  Version 2.2.0


In [67]:
# check and set path before loading modules
print(os.getcwd())
DIR = "/tf/notebooks/schnemau/xAI_stroke_3d/"
if os.getcwd() != DIR:
    os.chdir(DIR)
    print(os.getcwd())

import functions_metrics as fm
import functions_model_definition as md
import functions_read_data as rdat
import Utils_maurice as utils
import functions_gradcam as gc


#ontram functions
from k_ontram_functions.ontram import ontram
from k_ontram_functions.ontram_loss import ontram_loss
from k_ontram_functions.ontram_metrics import ontram_acc, ontram_auc
from k_ontram_functions.ontram_predict import predict_ontram, get_parameters

#weights tuning functions
import ens_weights_tuning as w_tune

/tf/notebooks/schnemau/xAI_stroke_3d
/tf/notebooks/schnemau/xAI_stroke_3d


In [96]:
selected_patient = 105
all_results = "/tf/notebooks/schnemau/xAI_stroke_3d/data/all_tab_results_10Fold_CIBLSX_M1.csv"
res = pd.read_csv(all_results, sep=",")
res = res[res['p_id'] == selected_patient]
res

,p_idx,p_id,mrs,unfavorable,fold0,fold1,fold2,fold3,fold4,fold5,...,threshold_avg,threshold_avg_w,y_pred_class_avg,y_pred_class_avg_w,y_pred_std,y_pred_unc,y_pred_std_w,y_pred_unc_w,pred_correct,pred_correct_w
90,73,105,6.0,1,train,train,test,train,train,train,...,0.247005,0.298656,0,0,0.109692,0.274989,0.102801,0.259854,False,False


In [116]:
dat_norm = pd.read_csv("/tf/notebooks/hezo/stroke_perfusion/data/baseline_data_zurich_prepared.csv", sep = ",")
tab = dat_norm[dat_norm['p_id'] == selected_patient]
tab

,p_id,mrs3,age,sexm,nihss_baseline,mrs_before,stroke_beforey,tia_beforey,ich_beforey,rf_hypertoniay,rf_diabetesy,rf_hypercholesterolemiay,rf_smokery,rf_atrial_fibrillationy,rf_chdy,eventtia
3,105,6,1.432992,-1.295848,-0.820817,-0.470965,-0.419507,-0.263596,-0.118205,0.712688,-0.448358,0.806338,-0.556297,-0.460976,-0.473492,0


In [134]:
tab_array = tab.values[0,2:]
tab_array

array([ 1.43299153, -1.29584762, -0.82081682, -0.47096543, -0.41950722,
       -0.26359616, -0.11820485,  0.71268841, -0.44835758,  0.80633793,
       -0.55629662, -0.46097582, -0.47349193,  0.        ])

In [135]:
selected_covariate = "age"
dat_norm = pd.read_csv("/tf/notebooks/hezo/stroke_perfusion/data/baseline_data_zurich_prepared.csv", sep = ",")
values = np.linspace(dat_norm[selected_covariate].min(), dat_norm[selected_covariate].max(), num=40)

In [138]:
covariate_index = 0
value = values[0]
modified_data = tab_array.copy()

In [141]:
data_modified = []
covariate_index = 0

for value in values:
    modified_data = tab_array.copy()
    modified_data[covariate_index] = value
    data_modified.append(modified_data)  # Assuming you want to visualize average predictions

In [100]:
X_in = np.load("/tf/notebooks/schnemau/xAI_stroke_3d/data/prepocessed_dicom_3d.npy")
imgs = np.expand_dims(X_in, axis = -1)
img_pat = imgs[res['p_idx'].values]
img_pat.shape

(1, 128, 128, 28, 1)

In [102]:
test_index = (res == 'test').any().index[(res == 'test').any()].tolist()[0]
which_split = int(test_index[-1])
which_split

2

In [62]:
version = "10Fold_CIBLSX" 
layer_connection = "globalAveragePooling" 
last_activation = "linear" 
# Define Model Version
model_version = 1
input_dim = (128, 128, 28, 1)
output_dim = 1
batch_size = 6
C = 2 
model_nr = 0

mbl = utils.img_model_linear_final(input_dim, output_dim)
mls = utils.mod_linear_shift(13)
model_3d = ontram(mbl, mls)             

model_name = ("/tf/notebooks/schnemau/xAI_stroke_3d/weights/10Fold_CIBLSX/3d_cnn_binary_model_split" + "CIB_LSX" + str(which_split) + 
                          "_normalized_avg_layer_paper_model_" + last_activation + 
                          "_activation_" + str(model_version) + "_" + str(model_nr) + ".h5")    

model_3d.load_weights(model_name)    
